# The Meataxe64 Package, Core Functionality

In [47]:
LoadPackage("meataxe64");; Read("../gap/bench.g"); 
Read("../gap/mtx64utils.g"); LoadPackage("jupyterviz");;

## Matrix Multiplication

The Classification of Finite Simple Groups, completed in the 1980s was one of the highlights of 20th Century mathematics. Part of it was the discovery of the 26 unique and reemarkable sporadic simple groups, of which the largest is the Fischer-Griess Monster $\mathbb{M}$ of order 

$$808017424794512875886459904961710757005754368000000000$$ 

The *most* tractable representation of this group is as $196882\times 196882$ matrices of integers mod 2. Each matrix is 5GB.

For many years this was clearly beyond any practical computation and computational study of this group has to use complex and limited indirect methods. The first actual multiplication of two dense elements was done in 1998, using most of the computing resources of a large maths department for 45 hours.

Before OpenDreamKit this computation would still have taken days on a powerful workstation. We have contributed to the development of a massively improved C and assembler library and a GAP interface for it.

It now takes about 8 minutes on a laptop, scaling to 2-3 minutes on a multicore server. That's a bit too much for a demo, so we'll drop to more convenient $100\, 000 \times 100\, 000$ matrices.


In [33]:
m := MTX64_RandomMat(MTX64_FiniteField(2),100000,100000);

< matrix 100000x100000 : <MTX64 GF(2)>>

We can multiply this by itself:

In [48]:
ShowBench(ParMult,m,m);

wall time: 63.9s cpu time: 606s memory allocated: 1.16GB result returned


## Gaussian Elimination

Our other key primitive operation. To see what it does properly we need a singular matrix. 

We take the Kronecker (tensor) product of two rectangular matrices. 

If $A$ is $n\times m$ and $B$ is $m\times n$ with $m < n$ then $A\otimes B$ will have rank at most $m^2$.

In [83]:
f := MTX64_FiniteField(9);;
m1 := MTX64_RandomMat(f, 200,99);;
m2 := MTX64_RandomMat(f, 99,200);;
m := MTX64_KroneckerProduct(m1,m2);

< matrix 19800x19800 : <MTX64 GF(3^2)>>

Our basic Gaussian elimnination operation applied to a matrix $A$, computes $M$, $K$, $R$, $\gamma$ and $\rho$ satisfying: 

$$\pmatrix{M&0\cr K & 1} \rho A \gamma = \pmatrix{-1&R\cr0&0}$$ 

where $\gamma$ and $\rho$ are permutations that effectively select the pivot columns and pivot rows of $A$. This is effectively full reduced row echelon form, with transforming matrices.

Using this, we can compute inverses, solve systems of equations, determine nullspaces, etc. efficiently.

In [88]:
ech := fail;; # suppress a warning.
ShowBench(function() ech := MTX64_Echelize(m);end); 
ech.multiplier; ech.cleaner; ech.remnant;  # M, K and R in the above formula

wall time: 11.7s cpu time: 11.1s memory allocated: 747.81MB no result returned


< matrix 9801x9801 : <MTX64 GF(3^2)>>

< matrix 9999x9801 : <MTX64 GF(3^2)>>

< matrix 9801x9999 : <MTX64 GF(3^2)>>

We an compare this runtime to multiplication. If $m$ was full rank we would expect them to be the same, since $m$ is lower rank, the Gaussian elimination is actually faster.

In [89]:
ShowBench(\*,m,m);

wall time: 24.8s cpu time: 24.6s memory allocated: 186.95MB result returned


We can also use the multi-threaded version of the Gaussian elimination

In [39]:
MTX64_WriteMatrix(m, "a"); 
ShowBench(MTX64_fEchelize, ".", "a", "gamma", "rho", "m", "k", "r");

wall time: 27.5s cpu time: 204s memory allocated: 144B result returned


true

## Run-time versus matrix size

We set the field and maximum dimension and make a set of random matrices of different sizes

In [75]:
q := 3;; maxdim := 20000;; 
sizes := List([1..16], i-> i*QuoInt(maxdim, 16));;

In [76]:
mats := List(sizes, i-> MTX64_RandomMat(MTX64_FiniteField(q), i, i));;

And look at the timing for squaring them:

In [79]:
marks1 := List(mats, x-> BenchMark(\*,x,x));;
marksm := List(mats, x-> BenchMark(ParMult,x,x));;
Plot(
[sizes,List(marks1, x-> x.cpu), rec(name := "single-threaded",
title := "Meataxe64 runtimes for matrix multiply", xaxis := "Dimension", yaxis := "ms")],
[sizes,List(marksm, x-> QuoInt(x.wall,10^6)), rec(name := "multi-threaded wall time")]
);
